In [1]:
using GaussianProcesses, Plots, SumProductNetworks, StatsFuns, Distributions
import SumProductNetworks.add!

#### Include own implementations

In [2]:
include("utilFunctions.jl")
include("dataTypes.jl")
include("dataTypeUtilFunctions.jl")
include("computationFunctions.jl")

spn_rand (generic function with 3 methods)

#### Data set

In [3]:
gr(size = (1024, 768))

Plots.GRBackend()

In [4]:
datapath = "../data/clean/motor.csv"

"../data/clean/motor.csv"

In [12]:
(data, header) = readcsv(datapath, header = true)

([2.4 0.0 1.0 3.7; 2.6 -1.3 1.0 3.7; … ; 55.4 -2.7 3.0 138.0; 57.6 10.7 3.0 138.0], AbstractString["times" "accel" "strata" "v"])

In [13]:
headerDict = Dict(col[2] => col[1] for col in enumerate(header))

Dict{SubString{String},Int64} with 4 entries:
  "v"      => 4
  "times"  => 1
  "accel"  => 2
  "strata" => 3

In [14]:
X = convert(Vector, data[:,headerDict["times"]])
y = convert(Vector, data[:,headerDict["accel"]]);

In [15]:
y /= maximum(y);

In [16]:
N = length(X)

94

In [17]:
scatter(X, y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

## Learn a full GP

In [18]:
mZero = MeanZero()                   #Zero mean function
kern = SE(0.0,0.0)                   #Sqaured exponential kernel (note that hyperparameters are on the log scale)

Type: GaussianProcesses.SEIso, Params: [0.0, 0.0]


In [19]:
logObsNoise = -1.0                        # log standard deviation of observation noise (this is optional)

-1.0

In [56]:
log(5)

1.6094379124341003

In [61]:
gp = GP(reshape(X, 1, N), y, MeanZero(), SE(log(5.0),log(1.0)), -2.)       #Fit the GP

GP Exact object:
  Dim = 1
  Number of observations = 94
  Mean function:
    Type: GaussianProcesses.MeanZero, Params: Float64[]
  Kernel:
    Type: GaussianProcesses.SEIso, Params: [1.60944, 0.0]
  Input observations = 
[2.4 2.6 … 55.4 57.6]
  Output observations = [0.0, -0.0173333, -0.036, 0.0, -0.036, -0.036, -0.036, -0.0173333, -0.036, -0.036  …  0.142667, 0.142667, -0.357333, -0.177333, 0.0, 0.142667, -0.196, -0.036, -0.036, 0.142667]
  Variance of observation noise = 0.01831563888873418
  Marginal Log-Likelihood = -125.001

In [54]:
μ, σ² = predict_y(gp,linspace(minimum(X),maximum(X),100));

In [55]:
scatter(X, y, label = "observations", title = "Full GP with SE Kernel and LLH: $(gp.target)")
plot!(linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "full GP with 95% interval")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 Full GP with SE Kernel and LLH: -36.41658636106447 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polygon clip-path="url(#clip2402)" points="
56.8551,412.775 66.1466,410.086 75.4382,409.133 84.7297,409.408 94.0213,410.355 103.313,411.515 112.604,412.595 121.896,413.443 131.188,413.988 140.479,414.17 
 149.771,413.912 159.062,413.121 168.354,411.733 177.645,409.768 186.937,407.369 196.228,404.83 205.52,402.607 214.812,401.308 224.103,401.682 233.395,404.576 
 242.686,410.864 251.978,421.34 261.269,436.584 270.561,456.832 279.852,481.87 289.144,510.998 298.436,543.061 307.727,576.564 317.019,609.827 326.31,641.144 
 335.602,668.94 344.893,691.868 354.185,708.882 363.476,719.263 372.768,722.629 382.059,718.924 391.351,708.41 400.643,691.628 409.934,669.358 419.226,642.555 
 428.517,612.276 437.809,579.608 447.1,545.616 456.392,511.294 465.683,477.552 474.975,445.202 484.267,414.964 493.558,387.462 502.85,363.219 512.141,342.652 
 521.433,326.055 530.724,313.587 540.016,305.265 549.307,300.966 558.599,300.444 567.891,303.352 577.182,309.257 586.474,317.642 595.765,327.899 605.057,339.319 
 614.348,351.109 623.64,362.442 632.931,372.551 642.223,380.828 651.515,386.902 660.806,390.672 670.098,392.291 679.389,392.11 688.681,390.613 697.972,388.351 
 707.264,385.905 716.555,383.849 725.847,382.717 735.139,382.939 744.43,384.783 753.722,388.293 763.013,393.275 772.305,399.323 781.596,405.9 790.888,412.429 
 800.179,418.387 809.471,423.37 818.763,427.121 828.054,429.534 837.346,430.636 846.637,430.56 855.929,429.517 865.22,427.76 874.512,425.549 883.803,423.116 
 893.095,420.641 902.387,418.238 911.678,415.974 920.97,413.891 930.261,412.054 939.553,410.578 948.844,409.655 958.136,409.534 967.427,410.48 976.719,412.707 
 976.719,127.195 967.427,133.619 958.136,139.017 948.844,143.402 939.553,146.9 930.261,149.709 920.97,152.049 911.678,154.131 902.387,156.13 893.095,158.167 
 883.803,160.294 874.512,162.478 865.22,164.602 855.929,166.461 846.637,167.792 837.346,168.311 828.054,167.773 818.763,166.032 809.471,163.089 800.179,159.106 
 790.888,154.389 781.596,149.34 772.305,144.395 763.013,139.966 753.722,136.401 744.43,133.95 735.139,132.747 725.847,132.786 716.555,133.897 707.264,135.739 
 697.972,137.823 688.681,139.574 679.389,140.42 670.098,139.891 660.806,137.678 651.515,133.657 642.223,127.861 632.931,120.445 623.64,111.645 614.348,101.777 
 605.057,91.2556 595.765,80.6195 586.474,70.5447 577.182,61.815 567.891,55.2541 558.599,51.6398 549.307,51.6261 540.016,55.6925 530.724,64.1228 521.433,77.0091 
 512.141,94.2667 502.85,115.654 493.558,140.788 484.267,169.167 474.975,200.178 465.683,233.117 456.392,267.201 447.1,301.582 437.809,335.363 428.517,367.61 
 419.226,397.369 409.934,423.677 400.643,445.591 391.351,462.216 382.059,472.759 372.768,476.604 363.476,473.395 354.185,463.119 344.893,446.157 335.602,423.295 
 326.31,395.682 317.019,364.729 307.727,331.993 298.436,299.044 289.144,267.351 279.852,238.194 270.561,212.605 261.269,191.319 251.978,174.748 242.686,162.954 
 233.395,155.653 224.103,152.245 214.812,151.891 205.52,153.62 196.228,156.46 186.937,159.552 177.645,162.239 168.354,164.103 159.062,164.959 149.771,164.821 
 140.479,163.861 131.188,162.36 121.896,160.664 112.604,159.106 103.313,157.928 94.0213,157.184 84.7297,156.678 75.4382,155.956 66.1466,154.393 56.8551,151.378 
 
 " fill="#e26f46" fill-opacity="0.5"/>
<polyline clip-path="url(#clip2402)" style="stroke:#e26f46; stroke-width:1; stroke-opacity

In [545]:
yhat = reduce(vcat, [rand(gp, linspace(minimum(X),maximum(X),100)) for k in 1:100]);
yhat += rand() * 0.2
size(yhat)

(10000,)

In [671]:
μ, σ = predict_y(gp,linspace(minimum(X),maximum(X),100));
yhat = reduce(hcat, [rand(gp, linspace(minimum(X),maximum(X),100)) for k in 1:100]);

scatter(X, y, label = "observations", title = "Full GP with SE Kernel and LLH: $(gp.target)")
plot!(reduce(hcat, [linspace(minimum(X),maximum(X),100) for k in 1:100]), yhat, markersize = 2, markerstrokewidth = 0, legend =false)
plot!(linspace(minimum(X),maximum(X),100), μ + 2*σ, color = :green, w = 4, label = "Full GP mean + 2*std")
plot!(linspace(minimum(X),maximum(X),100), μ - 2*σ, color = :green, w = 4, label = "Full GP mean - 2*std")
savefig("../plots/full_gp_draws.png")

## optimize a full GP

In [62]:
optimize!(gp)

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [-2.0,1.6094379124341003,0.0]
 * Minimizer: [-1.2282746156639472,1.6156825479807833, ...]
 * Minimum: 3.510034e+01
 * Iterations: 12
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-08: true
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 56
 * Gradient Calls: 56

In [63]:
gp.target

-35.100343169513074

In [198]:
μ, σ² = predict_y(gp,linspace(minimum(X),maximum(X),100));

In [199]:
scatter(X, y, label = "observations", title = "Marginal LLH optimized full GP with SE Kernel and LLH: $(gp.target)")
plot!(linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "optimized full GP with 95% interval")
savefig("../plots/optimized_fullGP.png")

# Make a SPN with GP leaves

In [22]:
global gID = 0

0

Hand-coded 2-layer example

In [557]:
srand(12345678)
K = 10;
numSamples = 0.25;
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;

In [558]:
root = GPSumNode(nextID(), Int[]);

for k in 1:K
    
    # make random (1D) split
    split = rand() * maximum(X) + minimum(X)
    n = min(sum(X .>= split), sum(X .< split))
    c = 0
    
    while n < (N * numSamples)
        @assert c < 100 "Could not find a split"
        split = rand() * maximum(X) + minimum(X)
        n = min(sum(X .>= split), sum(X .< split))
        c += 1
    end
    
    child = FiniteSplitNode(nextID(), Float64[split])
    
    # append two GP leaves
    leaf1 = GPLeaf{Any}(nextID(),
            GP(reshape(X[X .<= split], 1, sum(X .<= split)), y[X .<= split], meanFunction, kernelFunction, noise))
    leaf1.parents = SPNNode[]
    
    leaf2 = GPLeaf{Any}(nextID(),
            GP(reshape(X[X .> split], 1, sum(X .> split)), y[X .> split], meanFunction, kernelFunction, noise))
    leaf2.parents = SPNNode[]
    
    add!(child, leaf1)
    add!(child, leaf2)
    
    add!(root, child)
end

## Visualize splits

In [207]:
plt = scatter(X, y, label = "observations", title = "Random splits of SPN-GP")
plotSplits!(plt, getAllSplits(root))
savefig("../plots/randomSplits.png")

## Likelihood of product nodes

In [477]:

for (pi, pnode) in enumerate(root.children)
    
    μ, σ² = spn_predict(pnode, linspace(minimum(X),maximum(X),100));
    
    plt = scatter(X, y, label = "observations", title ="Prediction of product node $(pi), w: $(root.posterior_weights[pi]), LLH: $(spn_likelihood(pnode))")
    plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "MoE GP with 95% interval")
    plotSplits!(plt, getAllSplits(pnode))
    savefig("../plots/productNode$(pi).png")

end

## Compute likelihoods

In [209]:
# test
n = GPLeaf{Any}(0, GP(reshape(X, 1, N), y, meanFunction, kernelFunction, noise))
spn_likelihood(n)

-41.1350265676093

In [211]:
# test
n = root.children[3]
spn_likelihood(n)

-42.28561685154945

In [314]:
# test
spn_likelihood(root)

-42.830715836609755

## Update posterior weights

In [473]:
root.prior_weights = ones(length(root)) / length(root);

In [474]:
spn_update(root)

-42.830715836609755

In [476]:
println(root)

Gaussian Process Sum Node [ID: 2992175, 
	 w_prior: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
	 w_posterior: [0.034, 0.039, 0.172, 0.164, 0.108, 0.059, 0.053, 0.152, 0.108, 0.11]]


## Predictions under the model

In [223]:
# test
n = root.children[3]
μ, σ² = spn_predict(n, linspace(minimum(X),maximum(X),100));

plt = scatter(X, y, label = "observations", title ="Prediction of product node 3 only")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "MoE GP with 95% interval")
plotSplits!(plt, getAllSplits(n))
savefig("../plots/productNode3.png")

In [225]:
@assert sum(root.prior_weights) ≈ 1.
@assert sum(root.posterior_weights) ≈ 1.

In [313]:
# test
n = root
μ, σ² = spn_predict(n, linspace(minimum(X),maximum(X),100));

plt = scatter(X, y, label = "observations", title="SPN-GP with $(K) splits and LLH: $(spn_likelihood(root))")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "SPN-GP with 95% interval")
plotSplits!(plt, getAllSplits(n))
savefig("../plots/spn_gp.png")

## Print SPN

In [560]:
rDepth = depth(root)
for node in reverse(SumProductNetworks.order(root))
    print(repeat("\t", rDepth - depth(node)), " ", node, "\n")
end

 Gaussian Process Sum Node [ID: 3051227, 
	 w_prior: [0.002, 0.002, 0.004, 0.008, 0.016, 0.031, 0.062, 0.125, 0.25, 0.5], 
	 w_posterior: [0.002, 0.002, 0.004, 0.008, 0.016, 0.031, 0.062, 0.125, 0.25, 0.5]]
	 Split (Product) Node [ID: 3051255, split: [27.793]]
		 Gaussian Process Leaf Node [ID: 3051257, LLH: -22.783]
		 Gaussian Process Leaf Node [ID: 3051256, LLH: -19.95]
	 Split (Product) Node [ID: 3051252, split: [28.692]]
		 Gaussian Process Leaf Node [ID: 3051254, LLH: -20.958]
		 Gaussian Process Leaf Node [ID: 3051253, LLH: -21.797]
	 Split (Product) Node [ID: 3051249, split: [31.754]]
		 Gaussian Process Leaf Node [ID: 3051251, LLH: -17.589]
		 Gaussian Process Leaf Node [ID: 3051250, LLH: -24.821]
	 Split (Product) Node [ID: 3051246, split: [25.601]]
		 Gaussian Process Leaf Node [ID: 3051248, LLH: -28.753]
		 Gaussian Process Leaf Node [ID: 3051247, LLH: -14.707]
	 Split (Product) Node [ID: 3051243, split: [25.262]]
		 Gaussian Process Leaf Node [ID: 3051245, LLH: -29.492]
		

## Create SPN-GP with flexible depth

In [594]:
K = 5;
numSamples = 0.25;
minSamples = 40;
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;
maximumDepth = 1

param = SPNParameters(K, numSamples, minSamples, kernelFunction, meanFunction, noise)

root = makeSumNode(X, y, param, 1, maximumDepth)

Gaussian Process Sum Node [ID: 3067551, 
	 w_prior: [0.2, 0.2, 0.2, 0.2, 0.2], 
	 w_posterior: [0.2, 0.2, 0.2, 0.2, 0.2]]

In [595]:
spn_update(root)

-42.29047283482476

In [596]:
spn_likelihood(root)

-42.29047283482476

In [570]:
root

Gaussian Process Sum Node [ID: 3051258, 
	 w_prior: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
	 w_posterior: [0.181, 0.028, 0.08, 0.134, 0.176, 0.016, 0.026, 0.071, 0.198, 0.09]]

In [571]:
depth(root)

4

In [572]:
sum(isa.(SumProductNetworks.order(root), SumNode))

21

In [525]:
# test
n = root
μ, σ² = spn_predict(n, linspace(minimum(X),maximum(X),100));


plt = scatter(X, y, label = "observations", title="SPN-GP with depth $(depth(root)) and $(K) splits and LLH: $(spn_likelihood(root))")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "SPN-GP with 95% interval")
#plotSplits!(plt, getSplits(n, 1))
savefig("../plots/deep_spn_gp.png")

plt = scatter(X, y, label = "observations", title="SPN-GP with depth $(depth(root)) and $(K) splits and LLH: $(spn_likelihood(root))")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "SPN-GP with 95% interval")
plotSplits!(plt, getSplits(n, 1))
savefig("../plots/deep_spn_gp_splits.png")

In [500]:
include("../../infinitesumproductnetworks/src/plotSPN.jl")

spn2graphviz (generic function with 1 method)

In [552]:
print(root)

Gaussian Process Sum Node [ID: 3035894, 
	 w_prior: [0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02], 
	 w_posterior: [0.03, 0.03, 0.026, 0.019, 0.029, 0.007, 0.014, 0.012, 0.022, 0.007, 0.007, 0.009, 0.032, 0.028, 0.006, 0.017, 0.016, 0.007, 0.016, 0.023, 0.032, 0.021, 0.018, 0.028, 0.019, 0.085, 0.006, 0.008, 0.029, 0.007, 0.006, 0.018, 0.029, 0.03, 0.03, 0.026, 0.02, 0.007, 0.007, 0.03, 0.027, 0.021, 0.018, 0.006, 0.034, 0.027, 0.008, 0.007, 0.022, 0.017]]

## some plots

#### Flat SPN

In [652]:
K = 10000;
numSamples = 0.25;
minSamples = 40;
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;
maximumDepth = 1

param = SPNParameters(K, numSamples, minSamples, kernelFunction, meanFunction, noise)

root = makeSumNode(X, y, param, 1, maximumDepth)
spn_update(root)

-42.75693752401281

In [ ]:
for K in [5, 50, 500, 5_000, 10_000, 50_000]

    numSamples = 0.25;
    minSamples = 40;
    meanFunction = MeanZero();
    kernelFunction = SE(log(5.0),log(1.0));
    noise = -1.;
    maximumDepth = 3

    param = SPNParameters(K, numSamples, minSamples, kernelFunction, meanFunction, noise)

    root = makeSumNode(X, y, param, 1, maximumDepth)
    spn_update(root)

    S = 100

    x = linspace(minimum(X),maximum(X),100)
    xx = reduce(hcat, x for k in 1:S)
    yy = reduce(hcat, spn_rand(root, x) for k in 1:S)

    μ, σ = predict_y(gp,linspace(minimum(X),maximum(X),100));

    plt = scatter(X, y, label = "observations", legend = false)
    plot!(xx, yy, markersize = 2, markerstrokewidth = 0, label = "Posterior samples from SPN-GP");
    plot!(linspace(minimum(X),maximum(X),100), w =4, μ + 2*σ, color = :green, label = "Full GP mean + 2*std")
    plot!(linspace(minimum(X),maximum(X),100), w =4, μ - 2*σ, color = :green, label = "Full GP mean - 2*std")
    
    splits = [child.split[1] for child in children(root)]
    
    vline!([minimum(splits), maximum(splits)], color = :yellow, w = 4)

    plot(plt, size = (1024, 1024), title="SPN-GPs with differen $(K) splits")
    savefig("../plots/spn_gp_splits_$(K)_$(maximumDepth).png")
    
end

In [670]:
plot(
histogram([child.split[1] for child in children(root)], weightss = root.posterior_weights, title= "posterior weighted"), 
histogram([child.split[1] for child in children(root)], title = "prior over splits"))
savefig("../plots/split_weights.png")

#### Deep SPN

In [625]:
K = 5;
numSamples = 0.25;
minSamples = 40;
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;
maximumDepth = 100

param = SPNParameters(K, numSamples, minSamples, kernelFunction, meanFunction, noise)

root = makeSumNode(X, y, param, 1, maximumDepth)
spn_update(root)

-47.93831375543295

In [631]:
S = 100

x = linspace(minimum(X),maximum(X),100)
xx = reduce(hcat, x for k in 1:S)
yy = reduce(hcat, spn_rand(root, x) for k in 1:S)

μ, σ = predict_y(gp,linspace(minimum(X),maximum(X),100));

plt1 = scatter(X, y, label = "observations", legend = false)
plot!(xx, yy, markersize = 2, markerstrokewidth = 0, label = "Posterior samples from SPN-GP");
plot!(linspace(minimum(X),maximum(X),100), μ + 2*σ, color = :green, label = "Full GP mean + 2*std")
plot!(linspace(minimum(X),maximum(X),100), μ - 2*σ, color = :green, label = "Full GP mean - 2*std")

plt2 = scatter(X, y, label = "observations")
plot!(xx, yy, markersize = 2, markerstrokewidth = 0, label = "Posterior samples from SPN-GP");
plot!(linspace(minimum(X),maximum(X),100), μ + 2*σ, color = :green, label = "Full GP mean + 2*std")
plot!(linspace(minimum(X),maximum(X),100), μ - 2*σ, color = :green, label = "Full GP mean - 2*std")
plotSplits!(plt2, getAllSplits(root))

plot(plt1, plt2, size = (2024, 768), title="SPN-GP with depth $(depth(root) - 1) and $(K) splits per sum node (LLH: $(spn_likelihood(root)))")
savefig("../plots/spn_gp_deep.png")